In [1]:
# install octave
!sudo apt-get -qq update
!sudo apt-get -qq install octave octave-signal liboctave-dev

# install oct2py that compatible with colab
import os

from pkg_resources import get_distribution

os.system(
    f"pip install -qq"
    f" ipykernel=={get_distribution('ipykernel').version}"
    f" ipython=={get_distribution('ipython').version}"
    f" tornado=={get_distribution('tornado').version}"
    f" oct2py"
)

# install packages
!pip install -qq matpower matpowercaseframes

In [2]:
import matpower
import oct2py

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.7.2
matpower version: 8.0.0.2.2.2


In [3]:
from matpower import start_instance
from matpowercaseframes import CaseFrames

In [4]:
m = start_instance()

In [22]:
cf = CaseFrames("case9", load_case_engine=m)
cf.bus[["PD", "QD"]] = 0.2 * cf.bus[["PD", "QD"]]
mpopt = m.mpoption("verbose", 2)
m.push("_mpopt", mpopt)
m.push("_mpc", cf.to_mpc(), verbose=False)
m.eval("_r1 = runuopf(_mpc, _mpopt);", verbose=False)

In [6]:
# pull all after cleansing unsupported data
m.eval(
    """
    _r1.raw = rmfield(_r1.raw, 'task');
    _r1 = rmfield(_r1, 'om');
    """
)
mpc = m.pull("_r1")

In [7]:
# selective pull
r1_mpc = m.eval(
    "struct('baseMVA', _r1.baseMVA, 'version', _r1.version, 'bus', _r1.bus,"
    " 'gen', _r1.gen, 'branch', _r1.branch, 'gencost', _r1.gencost);"
)
cf = CaseFrames(r1_mpc)
cf.gen

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF,MU_PMAX,MU_PMIN,MU_QMAX,MU_QMIN
1,1.0,0.000000,0.000000,300.0,-300.0,1.058571,100.0,0.0,250.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,37.176981,-43.256980,300.0,-300.0,0.900079,100.0,1.0,300.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,26.620814,-46.898656,300.0,-300.0,0.900666,100.0,1.0,270.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
alg = m.eval("_r1.raw.output.alg;")
alg

'MIPS'

In [8]:
# # NOTE: you also can run like this
# r1_mpc = m.eval(
#     """
#     mpopt = mpoption('verbose', 2);
#     mpc = loadcase('case9');
#     _r1 = runuopf(mpc, mpopt);
#     """
#
#     "struct("
#     " 'baseMVA', _r1.baseMVA, 'version', _r1.version, 'bus', _r1.bus, 'gen', _r1.gen,"
#     " 'branch', _r1.branch, 'gencost', _r1.gencost);"
# )
# r1_mpc